In [2]:
import numpy as np
import sympy as sp

In [3]:
def criar_vetor_de_simbolos(tamanho):
    """
    Cria um vetor (lista) de símbolos SymPy com o tamanho especificado.

    Parameters:
    tamanho (int): O número de símbolos a serem criados.

    Returns:
    list: Lista contendo os símbolos criados.
    """
    # Cria os símbolos usando sp.symbols
    return sp.symbols(f'x1:{tamanho+1}')

In [4]:
sp.Matrix(np.array_split(criar_vetor_de_simbolos(4), 2))

Matrix([
[x1, x2],
[x3, x4]])

In [5]:
# Função para calcular o produto Kronecker considerando binário exclusivo
def kronecker_binary(vectors):
    result = np.array([1])  # Inicia com o elemento neutro da multiplicação
    for v in vectors:
        result = np.kron(result, v)
    return result

In [6]:
kronecker_binary([[1,0],[0,1]])

array([0, 1, 0, 0])

In [7]:
bits = 3
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(bits*2), bits)))
variaveis

Matrix([
[x1*x3*x5],
[x1*x3*x6],
[x1*x4*x5],
[x1*x4*x6],
[x2*x3*x5],
[x2*x3*x6],
[x2*x4*x5],
[x2*x4*x6]])

In [8]:
# Iterando pelo vetor com o índice e o valor
for linha, elemento in enumerate(variaveis):
    print(f"Linha {linha}: {elemento}")

Linha 0: x1*x3*x5
Linha 1: x1*x3*x6
Linha 2: x1*x4*x5
Linha 3: x1*x4*x6
Linha 4: x2*x3*x5
Linha 5: x2*x3*x6
Linha 6: x2*x4*x5
Linha 7: x2*x4*x6


In [9]:
def decimal_para_binario_vetor(numero_decimal, tamanho_bits=8):
    """
    Converte um número decimal para um vetor binário usando produtos de Kronecker.

    Args:
        numero_decimal (int): Número decimal a ser convertido.
        tamanho_bits (int, opcional): Número de bits para representar o número binário. Padrão é 8.

    Returns:
        np.ndarray: Vetor resultante da conversão.
    
    Raises:
        ValueError: Se o número decimal for negativo ou se o tamanho de bits for insuficiente.
    """
    # Mapeamento dos caracteres binários para os vetores desejados
    mapeamento = {'0': [1, 0], '1': [0, 1]}

    # Verifica se o número é um inteiro não negativo
    if not isinstance(numero_decimal, int):
        raise TypeError("O número decimal deve ser um inteiro.")
    if numero_decimal < 0:
        raise ValueError("A função não suporta números decimais negativos.")

    # Converter o número decimal para binário e remover o prefixo '0b'
    numero_binario = bin(numero_decimal)[2:]

    # Verifica se o número binário cabe no tamanho de bits desejado
    if len(numero_binario) > tamanho_bits:
        raise ValueError(f"O número binário '{numero_binario}' excede o tamanho de bits especificado ({tamanho_bits} bits).")

    # Adiciona zeros à esquerda para completar o tamanho de bits desejado
    numero_binario = numero_binario.zfill(tamanho_bits)

    # Inicializa o vetor com 1 para o produto de Kronecker
    vetor = np.array([1])
    
    # Aplica o produto de Kronecker para cada dígito binário
    for digito in numero_binario:
        vetor = np.kron(vetor, mapeamento[digito])

    return vetor

In [10]:
vetor_dec = [
	0,
	0,
	0,
	0,
	0,
	0,
	0,
	1,
]

vetores_binarios = []
tamanho_bits = 1

for numero in vetor_dec:
    try:
        vetor = decimal_para_binario_vetor(int(numero), tamanho_bits)
        vetores_binarios.append(vetor)
    except (TypeError, ValueError) as e:
        print(f"Erro ao converter o número {numero}: {e}")

vetores_binarios = np.array(vetores_binarios).T
sp.Matrix(vetores_binarios)

Matrix([
[1, 1, 1, 1, 1, 1, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 1]])

In [11]:
vetor_dec = np.array(vetor_dec).reshape(-1, 1)
vetor_dec

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

In [12]:
vetores_binarios = []

for comb in vetor_dec:
	vetor = kronecker_binary(np.array([decimal_para_binario_vetor(int(bit), 1) for bit in comb]))
	vetores_binarios.append(vetor)

vetores_binarios = np.array(vetores_binarios).T
sp.Matrix(vetores_binarios)

Matrix([
[1, 1, 1, 1, 1, 1, 1, 0],
[0, 0, 0, 0, 0, 0, 0, 1]])

In [13]:
regra = sp.Matrix(np.dot(vetores_binarios, variaveis)) # expressões simbólicas
regra

Matrix([
[x1*x3*x5 + x1*x3*x6 + x1*x4*x5 + x1*x4*x6 + x2*x3*x5 + x2*x3*x6 + x2*x4*x5],
[                                                                  x2*x4*x6]])

In [14]:
def criar_substituicoes(bits):
    simbolos = criar_vetor_de_simbolos(bits * 2)
    substituicoes = {simbolos[i+1]: 1-simbolos[i] for i in range(0, len(simbolos), 2)}
    return substituicoes

In [15]:
substituicoes = criar_substituicoes(3)
substituicoes

{x2: 1 - x1, x4: 1 - x3, x6: 1 - x5}

In [37]:
f = sp.simplify(regra.subs(substituicoes))
f

Matrix([
[x1*x3*x5 - x1*x3 - x1*x5 + x1 - x3*x5 + x3 + x5],
[                    -(x1 - 1)*(x3 - 1)*(x5 - 1)]])

In [26]:
variaveis[0].free_symbols

{x1, x3, x5}

In [41]:
f[0].free_symbols

{x1, x3, x5}

In [40]:
from itertools import product

# Passo 3: Gerar todas as combinações de 0 e 1 para as 3 variáveis
entradas = list(product([0, 1], repeat=3))
entradas

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [49]:
", ".join(str(s) for s in f[0].free_symbols)

'x5, x1, x3'

In [53]:
for i, e in enumerate(f):
    variaveis_ordenadas = sorted(e.free_symbols, key=lambda s: s.name)
    print(f"F{i+1}({", ".join(str(s) for s in variaveis_ordenadas)}) = {e}")
    print("-------------------------")
    for vals in entradas:
        valor = e.subs({var: vals[i] for i, var in enumerate(variaveis_ordenadas)})
        print(f"{vals[0]}  {vals[1]}  {vals[2]}  |    {valor}")

F1(x1, x3, x5) = x1*x3*x5 - x1*x3 - x1*x5 + x1 - x3*x5 + x3 + x5
-------------------------
0  0  0  |    0
0  0  1  |    1
0  1  0  |    1
0  1  1  |    1
1  0  0  |    1
1  0  1  |    1
1  1  0  |    1
1  1  1  |    1
F2(x1, x3, x5) = -(x1 - 1)*(x3 - 1)*(x5 - 1)
-------------------------
0  0  0  |    1
0  0  1  |    0
0  1  0  |    0
0  1  1  |    0
1  0  0  |    0
1  0  1  |    0
1  1  0  |    0
1  1  1  |    0


In [ ]:
x1, x3, x5 = sp.symbols('x1 x3 x5')

print("x1 x3 x5 | f(x1, x3, x5)")
print("-------------------------")
for vals in entradas:
    valor = f[0].subs({x1: vals[0], x3: vals[1], x5: vals[2]})
    valor = int(valor)  # Garante que o valor seja 0 ou 1
    print(f"{vals[0]}  {vals[1]}  {vals[2]}  |    {valor}")

x1 x3 x5 | f(x1, x3, x5)
-------------------------
0  0  0  |    0
0  0  1  |    1
0  1  0  |    1
0  1  1  |    1
1  0  0  |    1
1  0  1  |    1
1  1  0  |    1
1  1  1  |    1


In [13]:
# Iterando pelo vetor com o índice e o valor
for linha, elemento in enumerate(regra):
    print(f"Linha {linha}: {elemento}")

Linha 0: x1*x3*x5 + x1*x3*x6 + x1*x4*x5 + x1*x4*x6 + x2*x3*x5 + x2*x3*x6 + x2*x4*x5
Linha 1: x2*x4*x6


In [14]:
lista = regra.tolist()[0]
list(set([x for x in lista if lista.count(x) > 1]))

[]

In [15]:
np.dot(vetores_binarios, decimal_para_binario_vetor(2, 4))

ValueError: shapes (2,8) and (16,) not aligned: 8 (dim 1) != 16 (dim 0)

In [ ]:
from sympy.utilities.lambdify import lambdastr

#  Lista para armazenar as funções
funcoes = []

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(bits*2)

# Iterar sobre cada elemento do vetor simbólico e criar uma função
for expr in regra:
    func = sp.lambdify(variaveis, expr)
    funcoes.append(func)


In [ ]:
from sympy.utilities.lambdify import lambdastr

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(bits*2)

for expr in regra:
    print(lambdastr(variaveis, expr))

lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (x1*x3*x5*x8)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (x1*x4*x6*x8)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x1,x2,x3,x4,x5,x6,x7,x8: (0)
lambda x

In [ ]:
import itertools

# Usando produto cartesiano
combinacoes = list(itertools.product([0, 1], repeat=4))

for comb in combinacoes:
	# Converte cada bit da combinação para um vetor binário e achata em uma única dimensão
	valores = np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]).flatten()
	print(valores,[f(*valores) for f in funcoes])

[1 0 1 0 1 0 1 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1 0 1 0 1 0 0 1] [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1 0 1 0 0 1 1 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0